<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #5: TF-IDF
`Fecha de entrega: Marzo 25, 2021. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [73]:
import re
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

from nltk.stem.snowball import SnowballStemmer
spanishStemmer=SnowballStemmer("spanish")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.feature_extraction.text import CountVectorizer


# Punto 1: Pre-Procesamiento

- `[9 pts]` Leer el archivo `bob_esponja.csv` usando `pandas`
- `[9 pts]` Crear una nueva columna con el texto en minúscula, sin caracteres especiales ni números, sin palabras vacias y hacer stemming de las palabras

In [74]:
dataxls = pd.read_csv("C:/Users/Memo/Desktop/npl/taller_5/bob_esponja.csv", index_col=0)
dataxls

,Descripción
Personaje,
Bob Esponja Pantalones Cuadrados,Bob Esponja es una esponja de mar con forma re...
Patricio Estrella,"Es una estrella de mar de color rosa, el mejor..."
Calamardo Tentáculos,Calamardo Tentáculos es un personaje principal...
Arenita Mejillas,Arenita Mejillas (Sandy Cheeks en inglés) en u...
Eugenio H. Cangrejo,Don Eugenio H. Cangrejo (normalmente llamado S...
Sheldon J. Plankton,"Sheldon J. Plankton, o simplemente Plankton, e..."
Karen Plankton,Karen Plankton es uno de los dos principales a...
Perlita Cangrejo,"Perla ""Perlita"" Cangrejo es un personaje princ..."
Sra. Puff,La Señora Puff es un personaje principal de Bo...


In [75]:
def txtProcesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords_sp]
    texto = " ".join(texto)
    return texto


def txtEstmm(texto):    
    #texto = texto.split()
    txtstem = []
    for pstring in texto.split():  
        stemm = spanishStemmer.stem(pstring)
        txtstem.append(stemm)
    return txtstem

def matTFIDF(texto):
    tfidf_vec = TfidfVectorizer()
    tfidf = tfidf_vec.fit_transform(texto)
    tfidf_matrix = pd.DataFrame(tfidf.toarray())
    tfidf_matrix.columns = tfidf_vec.get_feature_names()
    tfidf_matrix.index = data.index
    tfidf_matrix = tfidf_matrix.T.round(3)
    print(tfidf_matrix)

dataxls['procesado'] = dataxls['Descripción'].apply(lambda texto: txtProcesado(texto))
dataxls['stemming'] = dataxls['procesado'].apply(lambda texto: txtEstmm(texto))

ourlist = dataxls['stemming'].values.reshape(-1, 1)
steamdat = dataxls['stemming'].apply(lambda x: ' '.join(x))
steamdat

Personaje
Bob Esponja Pantalones Cuadrados    bob esponj esponj mar form rectangul color ama...
Patricio Estrella                   estrell mar color ros mejor amig bob esponj ju...
Calamardo Tentáculos                calamard tentacul personaj principal bob espon...
Arenita Mejillas                    arenit mejill sandy cheeks ingles personaj pri...
Eugenio H. Cangrejo                 don eugeni h cangrej normal llam señor cangrej...
Sheldon J. Plankton                 sheldon j plankton simplement plankton dos ant...
Karen Plankton                      kar plankton dos principal antagon bob esponj ...
Perlita Cangrejo                    perl perlit cangrej personaj principal bob esp...
Sra. Puff                           señor puff personaj principal bob esponj maest...
Gary el Caracol                     gary caracol personaj principal bob esponj que...
Name: stemming, dtype: object

# Punto 2: TF-IDF

- `[16 pts]` Crear la matriz TF-IDF

In [76]:
matTFIDF(steamdat)

Personaje  Bob Esponja Pantalones Cuadrados  Patricio Estrella  \
abaj                                  0.057               0.00   
aborrec                               0.000               0.00   
acab                                  0.000               0.00   
accident                              0.049               0.00   
acept                                 0.000               0.00   
...                                     ...                ...   
viv                                   0.000               0.05   
vol                                   0.057               0.00   
voz                                   0.195               0.00   
vuelv                                 0.172               0.00   
zapat                                 0.043               0.00   

Personaje  Calamardo Tentáculos  Arenita Mejillas  Eugenio H. Cangrejo  \
abaj                      0.000             0.000                0.000   
aborrec                   0.091             0.000          

# Punto 3: Distancia del coseno
- `[12 pts]` Calcular la distancia del coseno entre cada una de los personajes
- `[2 pts]` ¿Cuáles son los personajes más parecidos?
- `[2 pts]` ¿Cuáles son los personajes más diferentes?

In [77]:
from sklearn.metrics.pairwise import cosine_distances

dist_cos = cosine_distances(tfidf_matrix.T.values)
dist_cos = pd.DataFrame(dist_cos, columns = tfidf_matrix.columns, index = tfidf_matrix.columns)
dist_cos

Personaje,Bob Esponja Pantalones Cuadrados,Patricio Estrella,Calamardo Tentáculos,Arenita Mejillas,Eugenio H. Cangrejo,Sheldon J. Plankton,Karen Plankton,Perlita Cangrejo,Sra. Puff,Gary el Caracol
Personaje,,,,,,,,,,
Bob Esponja Pantalones Cuadrados,0.000000,0.826052,0.918644,0.838317,0.868204,0.872076,0.938609,0.885810,0.858077,0.924158
Patricio Estrella,0.826052,0.000000,0.790530,0.841122,0.831634,0.966449,0.930045,0.891502,0.952583,0.894790
Calamardo Tentáculos,0.918644,0.790530,0.000000,0.919857,0.831344,0.917606,0.949913,0.967265,0.968111,0.851644
Arenita Mejillas,0.838317,0.841122,0.919857,0.000000,0.908202,0.967774,0.948323,0.962015,0.972120,0.900641
Eugenio H. Cangrejo,0.868204,0.831634,0.831344,0.908202,0.000000,0.687380,0.891897,0.796723,0.896080,0.920664
Sheldon J. Plankton,0.872076,0.966449,0.917606,0.967774,0.687380,0.000000,0.794981,0.922577,0.986733,0.969239
Karen Plankton,0.938609,0.930045,0.949913,0.948323,0.891897,0.794981,0.000000,0.966088,0.984919,0.965252
Perlita Cangrejo,0.885810,0.891502,0.967265,0.962015,0.796723,0.922577,0.966088,0.000000,0.873832,0.969504
Sra. Puff,0.858077,0.952583,0.968111,0.972120,0.896080,0.986733,0.984919,0.873832,0.000000,0.964177


In [78]:
from sklearn.metrics.pairwise import euclidean_distances

dist_euc = euclidean_distances(tfidf_matrix.T.values)
dist_euc = pd.DataFrame(dist_euc, columns = tfidf_matrix.columns, index=tfidf_matrix.columns)

print(dist_euc)

Personaje                         Bob Esponja Pantalones Cuadrados  \
Personaje                                                            
Bob Esponja Pantalones Cuadrados                          0.000000   
Patricio Estrella                                         1.284040   
Calamardo Tentáculos                                      1.354393   
Arenita Mejillas                                          1.295001   
Eugenio H. Cangrejo                                       1.316485   
Sheldon J. Plankton                                       1.320922   
Karen Plankton                                            1.369698   
Perlita Cangrejo                                          1.330294   
Sra. Puff                                                 1.310513   
Gary el Caracol                                           1.357689   

Personaje                         Patricio Estrella  Calamardo Tentáculos  \
Personaje                                                                   
Bob E

# ¿Cuáles son los personajes más parecidos?

In [ ]:
bob esponja - patricio estrella 
patricia estrella - calamardo tentaculos
calamrado tentaculos - patricio estrella
arenita mejillas - bob esponja
eugenio h cangrejo - sheldon j plankton
sheldon j plankton - eugenio h cangrejo
karen plankton - sheldon j plankton
perlita cangrejo - eugenio h. cangrejo
sra puff - bob esponja 
gary el caracol - calamardo tentaculos

# ¿Cuáles son los personajes más diferentes?

In [ ]:
bob esponja - karen plankton
patricia estrella - sheldon j plankton
calamrado tentaculos - perlita cangrejo
arenita mejillas - sra puff
eugenio h cangrejo - sra puff
sheldon j plankton - sra puff
karen plankton - sra puff
perlita cangrejo - calamardo tentaculos
sra puff - karen plankton 
gary el caracol - sheldon j plankton